# How to train a NER to disambiguate homonyms with CPU

In this notebook, I will create and train a LLM with the aim of disambiguating homonyms thanks to Named-entity recognition (NER). This project is specifically for the Italian language but it is easily modifiable to adapt to other languages.


To get to the goal I will take advantage of one of spaCy's pre-trained models for natural language processing (https://spacy.io/usage/spacy-101)

In [ ]:
#let's install all the necessary for this project

!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download it_core_news_md
import spacy
from spacy.tokens import DocBin

#the following command will load an instance of the pre-trained model to do natural language processing
nlp = spacy.load("it_core_news_md")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 23.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In this specific case, I want to train the model to recognize when the word "pesca" (that in italian means both fishig and peach) is used in the context of sport or fruit. Therefore in the frase "Luigi ha vinto una gara di pesca sul lago mangiando una pesca matura" (Luigi won a lake fishing contest while eating a mature peach) it will have to recognize that the first "pesca" is the sport and the second one is the fruit

To train the model we need a database with some phrases where the word "pesca" is mapped to the correct context

In [ ]:
#lets create a list of phrases where pesca is a fruit
training_string_frutto = ["La composta di pesca è deliziosa spalmata su una fetta di pane tostato.",
                          "Per il brunch, ho preparato una macedonia con pesche, mirtilli e kiwi.",
                          "Le pesche grigliate con un filo di miele sono un contorno sorprendentemente buono per i piatti salati.",
                          "Mi piace fare uno scrub viso con pesca e zucchero per una pelle morbida e luminosa.",
                          "Ho aggiunto fette di pesca al mio yogurt per una colazione sana e saporita.",
                          "La crema pasticcera alla pesca è una variante interessante per farcire i dolci.",
                          "Durante l'estate, preparo sempre una crostata con pesche fresche del mercato.",
                          "Il profumo della pesca matura in cucina riempie la casa di un'aroma delizioso.",
                          "Le pesche essiccate sono uno snack dolce e naturale, perfetto da portare ovunque.",
                          "Ho preparato una salsa alla pesca per accompagnare il pollo alla griglia e ha avuto un grande successo.",
                          "In estate adoro mangiare una pesca fresca e succosa.",
                          "Il profumo dolce della pesca matura mi ricorda le vacanze in campagna.",
                          "Ho preparato una torta alla pesca per il dessert di stasera.",
                          "L'albero di pesca nel nostro giardino ha prodotto molti frutti quest'anno.",
                          "Il gelato alla pesca è uno dei miei gusti preferiti.",
                          "Durante il mercato, ho comprato un cesto di pesche appena raccolte.",
                          "La marmellata di pesca fatta in casa è perfetta per la colazione.",
                          "Tagliare una pesca a metà e riempirla di yogurt è uno snack delizioso e sano.",
                          "I succhi di frutta alla pesca sono molto rinfrescanti durante le giornate calde.",
                          "La buccia vellutata della pesca è così invitante che è difficile resistere a mangiarne una subito.",
                          "La crostata di pesca e mandorle è il dolce preferito della mia famiglia.",
                          "Ho assaggiato una pesca nettarina ed era incredibilmente dolce.",
                          "Le pesche sciroppate sono un ottimo modo per gustare questo frutto anche fuori stagione.",
                          "Durante il picnic, abbiamo portato una cesta piena di pesche e altri frutti estivi.",
                          "La pesca bianca ha un sapore più delicato rispetto alla pesca gialla.",
                          "Un'insalata con rucola, formaggio di capra e fette di pesca è un piatto estivo perfetto.",
                          "Le pesche arrostite con miele e cannella sono un dessert semplice ma delizioso.",
                          "Ho comprato una crema per il viso alla pesca e ha un profumo meraviglioso.",
                          "La salsa di pesche e peperoncino è un condimento interessante per carni grigliate.",
                          "Il frullato di pesca e banana è una bevanda nutriente e rinfrescante.",
                          "Il sorbetto alla pesca è un rinfrescante piacere estivo che non posso mai perdere.",
                          "Le pesche noci sono croccanti e deliziose, perfette per uno snack veloce.",
                          "Ho piantato un albero di pesca nel mio giardino e non vedo l'ora di raccogliere i frutti.",
                          "Le pesche in agrodolce si abbinano splendidamente ai piatti di carne.",
                          "Un centrifugato di pesca e menta è ideale per iniziare la giornata con energia.",
                          "Aggiungere pezzi di pesca al gelato alla vaniglia crea un dessert semplice e gustoso.",
                          "per merenda la cosa migliore è mangiare una pesca succosa",
                          "Le pesche caramellate sono un'ottima guarnizione per i pancake della colazione.",
                          "Ho trovato una ricetta di crumble alle pesche che è davvero irresistibile.",
                          "La pesca piatta, o saturnina, ha una forma particolare e un sapore molto dolce.",
                          "Un bicchiere di tè freddo alla pesca è perfetto per rinfrescarsi durante una giornata calda.",
                          "Le pesche mature sono una delizia da gustare direttamente dall'albero.",
                          "Ho comprato un cesto di pesche al mercato contadino per fare una marmellata.",
                          "La pesca è uno dei frutti più versatili in cucina, si presta a dolci e piatti salati.",
                          "I succhi di pesca sono una bevanda fresca e dissetante durante le giornate calde.",
                          "Preparare una crostata con le pesche appena raccolte è un vero piacere.",
                          "Le pesche bianche sono note per la loro polpa succosa e dolce.",
                          "il mio frutto preferito è la pesca matura",
                          "Con le pesche fresche si possono fare fantastiche insalate di frutta.",
                          "La salsa di pesche è un accompagnamento delizioso per carni grigliate.",
                          "Le pesche essiccate sono uno snack sano e gustoso da portare in giro.",
                          "La polpa della pesca si presta alla preparazione di gelati e sorbetti.",
                          "Ho piantato un albero di pesco nel mio giardino per avere frutta fresca ogni estate.",
                          "La confettura di pesche fatte in casa è un regalo apprezzato da tutti.",
                          "Le pesche sciroppate sono un ottimo modo per conservare il loro gusto anche fuori stagione.",
                          "Il succo di pesca fresco è una meravigliosa aggiunta alla colazione.",
                          "Le pesche mature profumano di estate e di dolcezza.",
                          "Le pesche cotte al forno con un po' di cannella sono un dessert confortante.",
                          "Il cesto di pesche mature profuma di estate e di dolcezza.",
                          "Le pesche mature sono così gustose che ne voglio sempre di più.",
                          "Le pesche mature sono un vero piacere per il palato.",
                          "La torta di pesche e mandorle è il dolce perfetto per una merenda estiva.",
                          "Le pesche mature racchiudono tutta la dolcezza dell'estate in un solo morso.",
                          "Le pesche mature sono un'esplosione di gusto e freschezza.",
                          "Ho raccolto una pesca dal mio albero e l'ho gustata appena colta.",
                          "Le pesche mature sono dolci e succose, un vero piacere per il palato.",
                          "La marmellata di pesca fatta in casa è perfetta per la colazione.",
                          "La pesca è uno dei frutti estivi più amati per la sua dolcezza e succosità.",
                          "Il gelato alla pesca è una deliziosa alternativa ai gusti più tradizionali.",
                          "una pesca matura profuma di estate e di dolcezza.",
                          "Le pesche appena raccolte sono così succose che è difficile mangiarne solo una.",
                          "Le pesche sciroppate sono un ottimo modo per conservare il loro gusto anche fuori stagione.",
                          "La torta di pesche e mandorle è un dolce tradizionale della mia regione.",
                          "La polpa della pesca si scioglie in bocca con ogni morso, è irresistibile.",
                          "Il succo di pesca fresco è una meravigliosa bevanda dissetante durante le giornate calde.",
                          "Le pesche mature sono così gustose che ne voglio sempre di più.",
                          "La salsa di pesche è un'ottima aggiunta per insaporire carni e piatti a base di pesce.",
                          "Le pesche mature sono un vero piacere per il palato, un'esplosione di gusto e freschezza.",
                          "Ho fatto una crostata con pesche e crema pasticcera, è divina!",
                          "Le pesche mature sono così succose che è difficile mangiarne una sola.",
                          "Il sorbetto alla pesca è un dessert fresco e leggero perfetto dopo un pasto abbondante.",
                          "La confettura di pesche fatte in casa è un regalo apprezzato da tutti gli amanti della colazione.",
                          "Le pesche mature racchiudono tutta la dolcezza dell'estate in un solo morso.",
                          "Ho preparato uno smoothie con pesche, yogurt e miele, è diventato il mio preferito.",
                          "Le fette di pesca cotte al forno con cannella e zucchero sono un dolce irresistibile.",
                          "Le pesche mature sono così succose che è difficile mangiarne solo una.",
                          "La torta di pesche e mandorle è un dessert perfetto da servire durante una cena speciale.",
                          "Il profumo di pesca maturo riempie la cucina di un'aroma delizioso.",
                          "Le pesche mature sono così gustose che è difficile resistere alla tentazione di mangiarne sempre di più.",
                          "Ho preparato una crostata di pesche e crema pasticcera per festeggiare l'arrivo dell'estate.",
                          "Le pesche mature sono così succose che ti fanno venire l'acquolina in bocca solo a guardarle.",
                          "Il gelato alla pesca fatto in casa è un'ottima alternativa per rinfrescarsi durante le giornate calde.",
                          "Le pesche mature sono così dolci e succose che si sciolgono in bocca con ogni morso.",
                          "Ho fatto una torta con pesche sciroppate e crema chantilly, è stata un successo!",
                          "Una fetta di pesca fresca è perfetta per iniziare la giornata.",
                          "La marmellata di pesca si abbina bene al pane tostato.",
                           "Il profumo della pesca matura riempie la cucina.",
                          "La mia torta preferita è quella con pesca e crema.",
                          "Il gelato alla pesca è sempre una scelta rinfrescante.",
                          "Il succo di pesca appena spremuto è delizioso.",
                          "Tagliare una pesca e aggiungerla allo yogurt è uno snack sano.",
                          "Il sapore della pesca è unico e inconfondibile.",
                          "La torta soffice con pesca è un dolce perfetto per ogni occasione.",
                          "La confettura di pesca si abbina bene ai formaggi stagionati.",
                          "Il gelato alla pesca con pezzi di frutta è una delizia.",
                          "La pesca è un ottimo ingrediente per preparare smoothie estivi.",
                          "Una salsa di pesca è un'ottima aggiunta per il gelato alla vaniglia.",
                          "Un bicchiere di succo di pesca fresco è sempre una buona idea.",
                          "La pesca tagliata a dadini è perfetta per decorare le torte.",
                          "Il crumble di pesca è un dolce croccante e gustoso.",
                          "Letizia guidava mangiando una pesca",
                          "La pesca al forno con zucchero e cannella è un dessert semplice ma delizioso.",
                          "Una mousse alla pesca è un dolce leggero e raffinato.",
                          "Il sapore della pesca si combina bene con il basilico fresco.",
                          "Una pesca matura è così succosa che il succo ti cola sulle mani.",
                          "Le fette di pesca sono ideali per una colazione leggera.",
                          "La pesca caramellata è un dessert irresistibile.",
                          "ho camminato per strada mentre mangiavo una pesca",
                          "Le fettine di pesca aggiungono colore e sapore alle insalate.",
                          "Il profumo della pesca matura è inconfondibile.",
                          "Una crema alla pesca è un'ottima farcitura per i dolci.",
                          "Il sapore della pesca è perfetto per i cocktail estivi.",
                          "La pesca è un frutto versatile che si adatta a molte ricette.",
                          "Un bicchiere di tè freddo alla pesca è perfetto per le giornate calde.",
                          "La pesca sciroppata è un'aggiunta gustosa ai dessert.",
                          "Una pesca matura è un'esplosione di dolcezza.",
                          "Il gelato alla pesca con pezzi di frutta è una vera delizia.",
                          "La pesca essiccata è un ottimo snack da portare con sé.",
                          "La salsa di pesca è perfetta per accompagnare i piatti di carne.",
                          "La pesca è un frutto che si presta a molte preparazioni culinarie.",
                          "Il profumo di una pesca appena tagliata è irresistibile.",
    "Adoro la dolcezza e la succosità di una pesca matura.",
    "La pesca è un frutto versatile che si può gustare al naturale, in frullati, torte o marmellate.",
    "Niente di meglio di una pesca fresca colta direttamente dall'albero.",
    "La pesca è un frutto ricco di vitamine e minerali, perfetto per una pausa salutare.",
    "Mi piace preparare la macedonia con pesche, melone e uva, un dessert fresco e rinfrescante.",
    "La pesca è un ingrediente delizioso per crostate, torte e muffin.",
    "Il succo di pesca è una bevanda dissetante e ricca di gusto.",
    "La pesca sciroppata è un dessert classico e sempre apprezzato.",
    "La confettura di pesca è perfetta per accompagnare pane tostato o biscotti.",
    "La pesca è un frutto che mi ricorda l'estate e le giornate al mare.",
    "Mi piace mangiare la pesca tagliata a spicchi con un filo di miele.",
    "La pesca è un frutto ottimo da portare in spiaggia o in gita.",
    "Un gelato alla pesca è il modo perfetto per concludere un pasto estivo.",
    "La pesca è un frutto che piace sia ai grandi che ai bambini.",
    "Spesso uso la pesca per preparare frullati freschi e nutrienti.",
    "La pesca è un ingrediente che aggiunge un tocco di dolcezza alle insalate.",
    "Mi piace preparare la salsa agrodolce con la pesca per accompagnare piatti di carne o pesce.",
    "La pesca è un frutto versatile che si può utilizzare in mille modi in cucina.",
    "Non c'è niente di meglio di una pesca fresca per una sana e gustosa merenda.",
    "La pesca è un frutto che mi fa stare bene e mi dà energia.",
    "Mangiare una pesca è come fare un viaggio sensoriale tra profumi e sapori.",
    "La pesca è un dono prezioso della natura che dovremmo apprezzare sempre.",
    "Ogni morso di pesca è un'esplosione di gusto e di freschezza.",
    "La pesca è un frutto che mi ricorda i bei ricordi dell'infanzia.",
    "Niente di meglio di una pesca fresca per affrontare una giornata calda.",
    "La pesca è un frutto che mi fa sentire felice e spensierata.",
    "Ogni morso di pesca è un viaggio verso il paradiso del gusto.",
    "La pesca è un frutto che mi fa sentire grata per la vita.",
    "Ogni volta che guardo una pesca mi stupisco della sua bellezza.",
    "La pesca è un frutto che mi ispira a essere migliore.",
    "Mangiare una pesca è come fare un piccolo gesto d'amore per me stessa.",
                          "Una fetta di pesca è un'ottima aggiunta al mio yogurt mattutino.",
    "Ho fatto una crostata con pesca e crema pasticcera, è deliziosa!",
    "Il profumo della pesca matura riempie tutta la casa.",
    "Il succo di pesca fresco è ideale per iniziare la giornata.",
    "La marmellata di pesca fatta in casa è perfetta per la colazione.",
    "Una pesca matura è così dolce e succosa che non riesco a resisterle.",
    "Il gelato alla pesca è il mio dessert preferito d'estate.",
    "Una pesca tagliata a dadini è ottima nelle insalate di frutta.",
    "Il sapore della pesca si abbina bene ai formaggi freschi.",
    "Una pesca matura è un piacere da gustare in qualsiasi momento.",
    "La salsa di pesca aggiunge un tocco di dolcezza ai piatti di carne.",
    "Un bicchiere di succo di pesca è perfetto per rinfrescarsi.",
    "La polpa della pesca si scioglie in bocca con ogni morso.",
    "Una pesca matura è il complemento ideale per una macedonia.",
    "Le fette di pesca sono perfette per decorare torte e dolci.",
    "Una pesca matura è una dolce tentazione che non si può ignorare.",
    "Il crumble di pesca è un dolce che piace a tutti.",
    "Una centrifuga di pesca e zenzero è molto rinfrescante.",
    "Una mousse di pesca è un dolce leggero e delizioso.",
    "La confettura di pesca è ottima sulle fette biscottate.",
    "Una pesca matura è l'ideale per una merenda sana.",
    "La pesca al forno con un po' di zucchero e cannella è squisita.",
    "Un'insalata di rucola, formaggio e pesca è molto gustosa.",
    "La pesca grigliata con miele è un dessert semplice ma delizioso.",
    "Il gelato alla pesca fatto in casa è una vera prelibatezza.",
    "Una cheesecake alla pesca è un dolce cremoso e irresistibile.",
    "Il sapore della pesca è perfetto per i frullati estivi.",
    "Una pesca matura è così succosa che il suo succo ti cola sulle mani.",
    "Il tè freddo alla pesca è una bevanda molto rinfrescante.",
    "La pesca caramellata è un dessert che conquista tutti.",
    "Una gelatina di pesca è un dolce fresco e leggero.",
    "Le fettine di pesca sono ideali per uno spuntino veloce.",
    "Il profumo della pesca matura è inebriante.",
    "Una crema di pesca è perfetta per farcire i dolci.",
    "La pesca in agrodolce è un contorno insolito ma gustoso.",
    "Il sorbetto alla pesca è un dessert perfetto per le giornate calde.",
    "La pesca essiccata è uno snack naturale e delizioso.",
    "Una pesca matura è un'esplosione di sapore in bocca.",
    "Il sapore della pesca è così delizioso che è difficile resisterle."
                          ]

#lets create a list of phrases where pesca is a sport
training_string_sport = ["Durante il weekend, Marco e i suoi amici sono andati a praticare la pesca sportiva sul lago.",
                         "La pesca d'altura richiede una buona preparazione fisica e attrezzature specifiche.",
                         "Ho imparato l'arte della pesca da mio nonno, che mi portava sempre al fiume da bambino.",
                         "Le gare di pesca sono molto popolari in questa regione, soprattutto in estate.",
                         "La pesca alla trota in torrente è una delle mie attività preferite per rilassarmi.",
                         "Esistono diverse tecniche di pesca sportiva, tra cui il fly fishing e lo spinning.",
                         "Per migliorare le tue abilità nella pesca, è utile frequentare dei corsi specifici.",
                         "Il torneo di pesca di quest'anno ha attirato partecipanti da tutto il paese.",
                         "Prima di andare a pesca, è importante controllare le previsioni meteo e le maree.",
                         "La pesca al lago offre la possibilità di catturare pesci di grandi dimensioni.",
                         "marco e anna sono andati a fare una gara di pesca a Genova",
                         "La pesca al colpo richiede molta pazienza e una buona conoscenza del comportamento dei pesci.",
                         "in esate molte persone vanno a fare pesca sul lago",
                         "Marta ha vinto il primo premio nel campionato regionale di pesca al black bass.",
                         "La pesca con esche artificiali è diventata molto popolare tra i pescatori sportivi.",
                         "Durante la vacanza in montagna, abbiamo dedicato un'intera giornata alla pesca nel lago alpino.",
                         "La pesca catch and release è una pratica comune per preservare le popolazioni ittiche.",
                         "Ogni anno, la nostra città organizza una festa dedicata alla pesca, con eventi e competizioni per tutti.",
                         "La pesca in mare aperto offre sfide uniche e la possibilità di catturare pesci di grandi dimensioni.",
                         "I bambini si sono divertiti moltissimo alla lezione di pesca organizzata dal club locale.",
                         "a mio figlio insegnerò larte della pesca sul lago",
                         "Il negozio di articoli per la pesca vicino al porto ha tutto ciò di cui un pescatore può aver bisogno.",
                         "La pesca in notturna può essere un'esperienza affascinante, soprattutto durante le notti estive.",
                         "La pesca sportiva è un'attività rilassante che permette di godere della natura.",
                         "Partecipare a una gara di pesca può essere emozionante e competitivo.",
                         "La pesca al salmone nei fiumi del nord è una delle esperienze più ambite dai pescatori.",
                         "Molti appassionati di pesca sportiva si dedicano alla pesca no-kill per preservare le specie ittiche.",
                         "Il miglior momento per la pesca al luccio è all'alba, quando i pesci sono più attivi.",
                         "La pesca in kayak sta diventando sempre più popolare tra gli amanti dell'avventura.",
                         "I laghi artificiali sono spesso usati per le competizioni di pesca sportiva.",
                         "La pesca in apnea richiede abilità particolari e una buona preparazione fisica.",
                         "Durante le vacanze estive, organizziamo spesso battute di pesca notturna.",
                         "Le canne da pesca di alta qualità possono fare la differenza nel successo di una battuta.",
                         "La pesca invernale sul ghiaccio è una tradizione in molti paesi nordici.",
                         "I pescatori sportivi devono spesso rispettare rigorose regolamentazioni per proteggere l'ecosistema.",
                         "La pesca di carpe giganti è una sfida che attira molti pescatori esperti.",
                         "Ho partecipato a un workshop di pesca sportiva per migliorare le mie tecniche.",
                         "La pesca in acque salate offre la possibilità di catturare una grande varietà di specie.",
                         "La scelta dell'esca giusta è cruciale per avere successo nella pesca.",
                         "Il club di pesca locale organizza eventi settimanali per i suoi membri.",
                         "La pesca alla traina è una tecnica efficace per catturare pesci predatori.",
                         "Gli appassionati di pesca sportiva amano condividere le loro esperienze e consigli.",
                         "La pesca dal molo è un'attività divertente e accessibile per tutti.",
                         "Gli accessori per la pesca, come i mulinelli e le esche, possono essere personalizzati.",
                         "La pesca in fiume richiede una buona conoscenza delle correnti e dei punti migliori.",
                         "Molti pescatori tengono un diario per annotare le loro catture e le condizioni meteo.",
                         "La pesca al bass è popolare soprattutto negli Stati Uniti e richiede tecniche specifiche.",
                         "Le barche da pesca sportiva sono dotate di tecnologie avanzate per individuare i pesci.",
                         "Le riserve di pesca sono aree protette dove è possibile praticare la pesca sportiva in modo sostenibile.",
                         "La pesca alla carpa si pratica spesso in laghetti o stagni appositamente attrezzati.",
                         "I bambini si divertono moltissimo durante le giornate di pesca organizzate per loro.",
                         "Le riviste di pesca sportiva offrono consigli utili e racconti di avventure da tutto il mondo.",
                         "La pesca sportiva è un'attività rilassante che permette di godere della natura.",
                         "Partecipare a una gara di pesca può essere emozionante e competitivo.",
                         "La pesca al salmone nei fiumi del nord è una delle esperienze più ambite dai pescatori.",
                         "Molti appassionati di pesca sportiva si dedicano alla pesca no-kill per preservare le specie ittiche.",
                         "Il miglior momento per la pesca al luccio è all'alba, quando i pesci sono più attivi.",
                         "La pesca in kayak sta diventando sempre più popolare tra gli amanti dell'avventura.",
                         "I laghi artificiali sono spesso usati per le competizioni di pesca sportiva.",
                         "Durante le vacanze estive, organizziamo spesso battute di pesca notturna.",
                         "Le canne da pesca di alta qualità possono fare la differenza nel successo di una battuta.",
                         "La pesca invernale sul ghiaccio è una tradizione in molti paesi nordici.",
                         "La pesca a mosca è considerata un'arte per la sua tecnica raffinata.",
                         "I pescatori sportivi devono spesso rispettare rigorose regolamentazioni per proteggere l'ecosistema.",
                         "La pesca di carpe giganti è una sfida che attira molti pescatori esperti.",
                         "Ho partecipato a un workshop di pesca sportiva per migliorare le mie tecniche.",
                         "La pesca in acque salate offre la possibilità di catturare una grande varietà di specie.",
                         "La scelta dell'esca giusta è cruciale per avere successo nella pesca.",
                         "Il club di pesca locale organizza eventi settimanali per i suoi membri.",
                         "Gli appassionati di pesca sportiva amano condividere le loro esperienze e consigli.",
                         "La pesca dal molo è un'attività divertente e accessibile per tutti.",
                         "Gli accessori per la pesca, come i mulinelli e le esche, possono essere personalizzati.",
                         "La pesca in fiume richiede una buona conoscenza delle correnti e dei punti migliori.",
                         "Molti pescatori tengono un diario per annotare le loro catture e le condizioni meteo.",
                         "La pesca al bass è popolare soprattutto negli Stati Uniti e richiede tecniche specifiche.",
                         "Le barche da pesca sportiva sono dotate di tecnologie avanzate per individuare i pesci.",
                         "Le riserve di pesca sono aree protette dove è possibile praticare la pesca sportiva in modo sostenibile.",
                         "La pesca alla carpa si pratica spesso in laghetti o stagni appositamente attrezzati.",
                         "I bambini si divertono moltissimo durante le giornate di pesca organizzate per loro.",
                         "Le riviste di pesca sportiva offrono consigli utili e racconti di avventure da tutto il mondo.",
                         "La pesca è un'attività rilassante che mi permette di stare a contatto con la natura.",
"Adoro la sensazione di attesa quando la lenza è in acqua e non so cosa potrò tirare su.",
"La pesca è uno sport che richiede pazienza, perseveranza e abilità.",
"Non c'è niente di più bello che catturare un pesce dopo una lunga giornata di pesca.",
"La pesca è un'ottima occasione per trascorrere del tempo con la famiglia e gli amici.",
"Mi piace insegnare la pesca ai bambini e vedere la loro emozione quando catturano il loro primo pesce.",
"La pesca è uno sport che mi permette di esplorare nuovi luoghi e di scoprire nuovi pesci.",
"Mi piace gareggiare in competizioni di pesca e mettere alla prova le mie abilità contro altri pescatori.",
"La pesca è un modo per staccare dalla routine quotidiana e dallo stress della vita moderna.",
"La pesca è una passione che mi accompagnerà per tutta la vita.",
                         "La pesca è uno sport che richiede molta pazienza e precisione.",
    "Ogni weekend vado a pesca con i miei amici lungo il fiume.",
    "Ho comprato una nuova canna da pesca per la mia prossima avventura.",
    "La pesca in mare aperto è un'esperienza emozionante e rilassante.",
    "Il mio nonno mi ha insegnato tutti i segreti della pesca.",
    "La pesca notturna può essere molto tranquilla e rilassante.",
    "Mi piace molto la pesca perché mi permette di stare a contatto con la natura.",
    "La pesca sportiva è un'attività molto diffusa nella nostra regione.",
    "Ho partecipato a una gara di pesca e ho vinto il primo premio.",
    "La pesca in acque dolci è diversa dalla pesca in mare, ma entrambe sono divertenti.",
    "Durante le vacanze estive, la pesca è il mio passatempo preferito.",
    "La pesca con la mosca richiede molta abilità e pratica.",
    "Ho preso il mio primo pesce durante una sessione di pesca al lago.",
    "La pesca a mosca è un tipo di pesca molto elegante e tecnico.",
    "La pesca dal molo è perfetta per chi ama la tranquillità.",
    "La pesca su una barca in mezzo al lago è estremamente rilassante.",
    "Mi sono appassionato alla pesca fin da bambino, grazie a mio padre.",
    "La pesca in kayak è un'esperienza unica e avventurosa.",
    "La pesca sul ghiaccio è molto popolare nei paesi nordici.",
    "La pesca è uno sport che ti insegna la pazienza e la perseveranza.",
    "Ho catturato un enorme pesce durante la mia ultima battuta di pesca.",
    "Il club di pesca locale organizza eventi e competizioni tutto l'anno.",
    "La pesca con l'amo e lenza è una tecnica molto antica.",
    "Mi piace esplorare nuovi luoghi per la pesca durante i miei viaggi.",
    "La pesca richiede una buona conoscenza delle abitudini dei pesci.",
    "La pesca è un ottimo modo per trascorrere del tempo all'aria aperta.",
    "Ho trascorso l'intera giornata a pesca, godendomi la tranquillità del lago.",
    "La pesca in solitaria mi permette di riflettere e rilassarmi.",
    "Ogni volta che vado a pesca, torno a casa con nuove storie da raccontare.",
    "La pesca d'altura può essere molto impegnativa ma anche gratificante.",
    "La pesca al largo della costa è perfetta per catturare pesci di grandi dimensioni.",
    "Il mio hobby preferito è la pesca, soprattutto nelle acque cristalline delle montagne.",
    "La pesca mi aiuta a staccare la spina e a ritrovare la serenità.",
    "Ho imparato molte tecniche di pesca grazie ai miei amici esperti.",
    "La pesca in torrente è un'attività molto rilassante e appagante.",
    "Ho trascorso un'intera settimana dedicata alla pesca in un lago isolato.",
    "La pesca subacquea è un'esperienza che richiede coraggio e abilità.",
    "Durante le vacanze, la pesca è la mia attività preferita per rilassarmi.",
    "Il piacere della pesca sta nel godersi ogni momento, dalla preparazione all'attesa.",
    "Ho acquistato una nuova attrezzatura per migliorare le mie abilità nella pesca.",
                         "Ho comprato una nuova canna da pesca per migliorare le mie prestazioni.",
    "La pesca è una delle attività più antiche praticate dall'uomo.",
    "La pesca in acque profonde richiede una buona preparazione fisica.",
    "Il mio hobby preferito è la pesca, soprattutto durante i mesi estivi.",
    "La pesca in kayak combina l'amore per l'acqua e la passione per lo sport.",
    "La pesca d'altura può regalare grandi soddisfazioni e catture memorabili.",
    "La pesca a mosca è considerata un'arte da molti appassionati.",
    "La pesca subacquea è un'esperienza affascinante e avventurosa.",
    "La pesca al salmone è molto popolare nei fiumi del nord.",
    "Ho trascorso un'intera giornata a pesca, senza rendermi conto del tempo che passava.",
    "La pesca in un lago di montagna è un'esperienza unica e rilassante.",
    "La pesca richiede una buona conoscenza delle condizioni meteorologiche.",
    "La pesca è un'ottima attività per rafforzare i legami familiari.",
    "Mi piace esplorare nuovi luoghi di pesca durante le vacanze.",
    "La pesca è uno sport che può essere praticato a qualsiasi età.",
    "Ho catturato il mio primo pesce quando avevo solo cinque anni.",
    "La pesca è un'attività perfetta per sfuggire allo stress quotidiano.",
    "Ho partecipato a una gara di pesca e ho avuto un grande successo.",
    "La pesca è uno sport che richiede calma e concentrazione.",
                         "Ogni anno partecipo a un torneo di pesca locale.",
    "La pesca sul fiume è una delle attività preferite della mia famiglia.",
    "Ho preso una giornata di ferie per dedicarmi alla pesca."


                         ]


parola = "pesca"

training_list = []

#we need to filter the lists to avoid having sentences where there is no "pesca" and we map the word to the correct context
for string in training_string_frutto:
    if parola in string:
        inizio = string.find(parola)
        fine = inizio + len(parola)
        training_list.append((string, [(inizio, fine, "frutto")]))
print(len(training_list))


for string in training_string_sport:
    if parola in string:
        inizio = string.find(parola)
        fine = inizio + len(parola)
        training_list.append((string,[(inizio, fine, "sport")]))
print(len(training_list))

#now we can create the actual database for the training and save it in the file train.spacy
db = DocBin()
for text, annotations in training_list:
    doc = nlp(text)
    ents = []
    tokens = []
    for token in doc:
        tokens.append(token)
    for start, end, label in annotations:
        span = doc.char_span(start, end, label=label)
        ents.append(span)
    if ents != [None]:
        doc.ents = ents
        db.add(doc)
db.to_disk("./train.spacy")

150
300


Now we have to download from spaCy's website a configuration file and add it to te files of the session (https://spacy.io/usage/training).
base_config.cfg includes all settings and hyperparameters for the training.
You can personalize it from the website or manually. In my case, I chose:
- language: Italian
- components: ner
- hardware: CPU
- optimize for: accuracy

Since I use the medium model for the italian language line 7 should be manually changed by putting "md" instead of "lg"


In [ ]:
#the next command will fill in the remaining defaults
!python -m spacy init fill-config base_config_CPU.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


It's time to train the model.<br>

In [ ]:
#The following command will create a new model and train it with the DB we have just created.
#This operation can take a long time. At the end, the new model will be saved in a new folder called "output".
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy

✔ Created output directory: output_150_CPU
ℹ Saving to output directory: output_150_CPU
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     45.00    0.00    0.00    0.00    0.00
  5     200          7.70    769.96   98.15   97.98   98.31    0.98
 11     400         10.16     42.96   98.82   98.98   98.65    0.99
 19     600         17.87     26.83  100.00  100.00  100.00    1.00
 29     800          0.00      0.00  100.00  100.00  100.00    1.00
 40    1000          0.00      0.00  100.00  100.00  100.00    1.00
 55    1200          0.00      0.00  100.00  100.00  100.00    1.00
 72    1400          0.00      0.00  100.00  100

⚠️⚠️⚠️<br>
Colab doesn't save files at the end of the session, so if you don't want to lose the model zip the folder and download it

In [ ]:
!zip -r output.zip ./output

  adding: output_150_CPU/ (stored 0%)
  adding: output_150_CPU/model-last/ (stored 0%)
  adding: output_150_CPU/model-last/meta.json (deflated 62%)
  adding: output_150_CPU/model-last/vocab/ (stored 0%)
  adding: output_150_CPU/model-last/vocab/lookups.bin (stored 0%)
  adding: output_150_CPU/model-last/vocab/vectors (deflated 9%)
  adding: output_150_CPU/model-last/vocab/key2row (deflated 8%)
  adding: output_150_CPU/model-last/vocab/vectors.cfg (stored 0%)
  adding: output_150_CPU/model-last/vocab/strings.json (deflated 78%)
  adding: output_150_CPU/model-last/config.cfg (deflated 61%)
  adding: output_150_CPU/model-last/tok2vec/ (stored 0%)
  adding: output_150_CPU/model-last/tok2vec/model (deflated 8%)
  adding: output_150_CPU/model-last/tok2vec/cfg (stored 0%)
  adding: output_150_CPU/model-last/tokenizer (deflated 85%)
  adding: output_150_CPU/model-last/ner/ (stored 0%)
  adding: output_150_CPU/model-last/ner/model (deflated 8%)
  adding: output_150_CPU/model-last/ner/cfg (defla

For the next time if you already have a train model you can just upload the model as a zip file in colab and then execute the command:

```
!unzip "./output.zip"
```
then you can continue from here as usual




We can now load the trained model and use it for natural language processing

In [ ]:
nlp1 = spacy.load("./output_150_CPU/model-best")

#Let's make some tests

In [ ]:
text = "luigi ha vinto una gara di pesca sul lago mangiando una pesca"
doc = nlp1(text)

In [ ]:
spacy.displacy.render(doc, style="ent")

In [ ]:
text = "per merenda prenderò una pesca"
doc = nlp1(text)
spacy.displacy.render(doc, style="ent")

In [ ]:
text = "al mercato compra due zucchine e una pesca"
doc = nlp1(text)
spacy.displacy.render(doc, style="ent")

In [ ]:
text = "le esche per la pesca sono molto costose"
doc = nlp1(text)
spacy.displacy.render(doc, style="ent")

In [ ]:
text = "ogni volta che simone va al mare pesca con gli amici"
doc = nlp1(text)
spacy.displacy.render(doc, style="ent")

In [ ]:
text = "al supermercato la pesca costa meno delle esche da pesca"
doc = nlp1(text)
spacy.displacy.render(doc, style="ent")

In [ ]:
tests = [
    "Dopo una lunga giornata di pesca al lago, ho gustato una dolce pesca fresca.",
    "Porto sempre una pesca con me quando vado a pesca per avere uno spuntino sano.",
    "Mentre aspettavo che il pesce abboccasse nella mia sessione di pesca, ho mangiato una pesca succosa.",
    "La mia merenda preferita durante le gare di pesca è una pesca matura.",
    "Ho preso il mio brevetto di pesca e poi ho festeggiato con una pesca dolcissima.",
    "Mentre mio padre era impegnato nella pesca, io mi godevo una pesca sotto l'ombrellone.",
    "Durante la pausa dalla pesca, mi sono rinfrescato con una pesca deliziosa.",
    "Il mio picnic perfetto include una pesca matura e una canna da pesca.",
    "Tra una battuta di pesca e l'altra, ho assaporato una pesca appena raccolta.",
    "La pesca sportiva richiede pazienza, così come aspettare che una pesca maturi."
]

for test in tests:
    doc = nlp1(test)
    spacy.displacy.render(doc, style="ent")

we can sew that this simple model has a good precision to understand the context of "pesca" when it has only one meaning in the sentence. <br>
When it appears with both meanings in the sentence, the model is less precise.<br>
To have better accuracy the size of the dataset can be increased, but this will comport a higher time and cost for training the model